In [ ]:
import os
import cv2
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D,MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.optimizers import RMSprop, SGD, Adam
from keras import regularizers
from keras.regularizers import l1

In [ ]:
num_classes = 6
img_rows, img_cols = 48, 48
batch_size = 512

#giving the training and testing data path
train_data_dir = './nxp/train/'
validation_data_dir = './nxp/validation/'

In [ ]:
#generating the image data using image data generator
val_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(
        rescale=1./255,
      rotation_range=30,
      shear_range=0.3,
      zoom_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

#we can use either the code above for image data generation or the below code for generation
#Also for generating the image data using image data generator(we can specify how many image we want to convert using one image)
#(using parameter like rotation range,shear range,zoom rangr,flip etc)
import os
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img

datagen = ImageDataGenerator( 
        rotation_range = 40, 
        shear_range = 0.4, 
        zoom_range = 0.3, 
        horizontal_flip = True, 
        brightness_range = (0.5, 1.5)) 


from PIL import Image
#path = "/home/bluebrain/Dynamic_Advertisement/nxp/train/"
a = ["Angry","Disgust","Happy","Neutral","Fear","Sad","Surprise"]
for i in a:
    dirs = os.listdir("/home/bluebrain/Dynamic_Advertisement/nxp/train/"+i)
    path = "/home/bluebrain/Dynamic_Advertisement/nxp/train/"+i
    for item in dirs:
        if item == ".DS_STORE":
            continue
        if os.path.isfile(path+item):
            img = Image.open(path+item)
            x = img_to_array(img) 
            x = x.reshape(x.shape)   
# using the above defined parameters.  
            j = 0
            for batch in datagen.flow(x, batch_size = 1, 
                          save_to_dir ='/home/bluebrain/Dynamic_Advertisement/nxp/train/'+i,  
                          save_prefix ='image', save_format ='jpeg'):
                j += 1
                if j > 4:
                    break

In [ ]:
#To remove checkpoints remove using(moving to the directory showing the checkpoints and remove by using the command
#"rm -r .ipynb_checkpoints"
print(validation_generator.class_indices)

In [ ]:
#architecture of model used
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',kernel_regularizer=regularizers.l2(0.0001),input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',kernel_regularizer=regularizers.l2(0.0001)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(7, kernel_size=(1, 1), activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Conv2D(7, kernel_size=(4, 4), activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Flatten())
model.add(Activation("softmax"))
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dense(6))
model.add(Activation('softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
nb_train_samples = 2227
nb_validation_samples = 408
epochs = 100
model_info = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples // batch_size,
            epochs=epochs,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples // batch_size)
model.save('model.h656')

In [ ]:
model.load_weights(os.path.join("model.h55"))

In [ ]:
from keras.models import load_model
classifier = load_model('model.h55')

In [ ]:
validation_generator = val_datagen.flow_from_directory(
        validation_data_dir,
        color_mode = 'grayscale',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
print(class_labels)

In [ ]:
#testing the model(running it on youtube video)
import cv2
import numpy as np
from time import sleep
from keras.preprocessing.image import img_to_array
import pafy
url = "https://www.youtube.com/watch?v=tiicx0d7yBg&t=20s"
my_video = pafy.new(url)
play = my_video.getbest(preftype = "mp4")

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img):
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return (0,0,0,0), np.zeros((48,48), np.uint8), img
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]

    try:
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation = cv2.INTER_AREA)
    except:
        return (x,w,y,h), np.zeros((48,48), np.uint8), img
    return (x,w,y,h), roi_gray, img

cap = cv2.VideoCapture(play.url)

while True:

    ret, frame = cap.read()
    rect, face, image = face_detector(frame)
    if np.sum([face]) != 0.0:
        roi = face.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

        # make a prediction on the ROI, then lookup the class
        preds = classifier.predict(roi)[0]
        label = class_labels[preds.argmax()]  
        label_position = (rect[0] + int((rect[1]/2)), rect[2] + 25)
        cv2.putText(image, label, label_position , cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0), 3)
    else:
        cv2.putText(image, "No Face Found", (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0), 3)
        
    cv2.imshow('All', image)
    ch = cv2.waitKey(1)
    if ch == 27 or ch == ord('q') or ch == ord('Q'):
        break
        
cap.release()
cv2.destroyAllWindows()                            